In [1]:
# Ajustar ruta a la raiz del proyecto
import re
import os
if not 'id_0123456789876543210' in locals():
    _rootlevel = 3
    _oldwd = re.sub(r'\\', '/', os.getcwd())
    _spdirs = _oldwd.split('/')
    _newwd = '/'.join(_spdirs[:(len(_spdirs)-_rootlevel)])
    os.chdir(_newwd)
    id_0123456789876543210 = None
print(f'Old WD: {_oldwd}')
print(f'New WD: {_newwd}')

Old WD: d:/Itzco/Documents/GitHub/BTC-LSTM/src/python/jnotes
New WD: d:/Itzco/Documents/GitHub/BTC-LSTM


In [4]:

from src.python.util.stats import data_transformer
from src.python.util.basic import load_arrow
from src.python.util.stats import histplot
from src.python.util.stats import sample
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# I Descripción de datos

A continuación, se describen las variables de las bases de datos del precio del bitcoin, descargados desde la plataforma binance:

- **Time**: La marca de tiempo (minutos) que indica el inicio del intervalo en el que se registra la información.

- **Open**: El precio de apertura del Bitcoin en un período de tiempo específico.

- **High**: El precio más alto alcanzado por el Bitcoin en ese mismo período.

- **Low**: El precio más bajo registrado para el Bitcoin durante el mismo período.

- **Close**: El precio de cierre del Bitcoin en el mismo período de tiempo.

- **Volume**: La cantidad total de Bitcoin negociada en el mercado durante el período.

- **VolumeUSDT**: La cantidad total en dólares estadounidenses (USDT) negociada durante el período.

- **TakerVolume**: La cantidad de Bitcoin intercambiada por los tomadores de mercado durante el período de tiempo especificado.

- **TakerVolumeUSDT**: La cantidad en dólares estadounidenses (USDT) intercambiada por los tomadores de mercado durante el mismo período de tiempo.

- **Trades**: El número total de transacciones o operaciones realizadas durante el período especificado.


**Nota**: Los tomadores de mercado son aquellos que toman las ofertas existentes por lo que aportan liquidez al mercado.

In [7]:
# carga de datos descargados en termporalidad de "30m"
data = load_arrow('data/01 download', 'BTCUSDT_30m')
data

,Time,Open,High,Low,Close,Volume,VolumeUSDT,TakerVolume,TakerVolumeUSDT,Trades
0,25049040.0,4261.48,4280.56,4261.32,4261.45,11.308926,4.822475e+04,3.936174,1.679304e+04,49
1,25049070.0,4280.00,4313.62,4267.99,4308.83,35.872083,1.541414e+05,31.224329,1.341594e+05,122
2,25049100.0,4308.83,4328.69,4304.31,4320.00,21.048648,9.086429e+04,19.396570,8.374608e+04,73
3,25049130.0,4320.00,4320.00,4291.37,4315.32,2.186268,9.440531e+03,2.051501,8.862196e+03,29
4,25049160.0,4330.29,4330.29,4309.37,4311.02,3.566277,1.540976e+04,2.302077,9.951673e+03,14
...,...,...,...,...,...,...,...,...,...,...
108219,28303860.0,34241.41,34446.42,34171.74,34252.91,1661.564360,5.697211e+07,658.524670,2.258272e+07,46309
108220,28303890.0,34252.91,34297.55,34170.05,34199.20,989.844540,3.386597e+07,424.350760,1.451775e+07,30004
108221,28303920.0,34199.20,34435.55,34199.19,34370.00,1173.322530,4.028753e+07,572.100420,1.964435e+07,37294
108222,28303950.0,34370.01,34417.36,34265.46,34337.01,993.382200,3.409540e+07,454.261440,1.558986e+07,28955


Los datos del precio de mercado se generan en tiempo continuo por lo que no deberían existir "huecos" en los mismos. Sin embargo, por razones desconocidas, existen algunos datos que no se encuentran registrados en los servidores de Binance. Esto podemos constatarlo con la variable **Time** ya que si existen saltos irregulares entre sus valores significaría que hay datos no registrados. Para este caso, en particular, los saltos de tiempo deben ser de 30 minutos:

In [78]:
index = np.where(data['Time'].diff() != 30)[0][1:]
print(f'Indices con saltos de tiempo:\n{index}')

empty = (np.array(data['Time'][index])-np.array(data['Time'][index-1]))/30
print(f'\nNúmero de datos faltantes en cada salto:\n{empty}')

print(f'\nTotal de datos faltantes:  {sum(empty)}')

print(f'\nLos datos faltantes representan un {round(100*sum(empty)/len(data),2)}% del los datos si registrados.')

Indices con saltos de tiempo:
[  985  5908  6704  8375 14935 14985 15293 20425 21658 27308 30370 31491
 34763 39067 39639 43283 43781 44437 46914 49981 57422 58444 58604 60910
 62008 64165 64405 69672 71929 97905]

Número de datos faltantes en cada salto:
[14.  3.  4. 68. 21.  4. 16.  8. 15. 13. 21.  2. 17.  5.  5.  3. 12.  5.
  6.  8.  3.  9.  3.  3.  4.  6.  9. 10.  5.  3.]

Total de datos faltantes:  305.0

Los datos faltantes representan un 0.28% del los datos si registrados.


Dicho lo anterior, los "huecos", que registran los datos, representan una proporción minúscula en comparación al tamaño total porque que resulta razonable estimar dicha información con base en los datos existentes. De manera general, para el caso de los precios, la estimación se realiza por medio de aproximaciones lineales entre los extremos de los datos existentes, en los casos de **High** y **Low** se añade, además de lo anterior, un factor de volatilidad proporcional a la cantidad de datos faltantes. Para estimar los volúmenes de operación se utiliza el promedio de los datos previos registrados con una ventana igual al tamaño de datos faltantes.

En el archivo `data/info/BLOCK_BTCUSDT_30m.csv` se encuentra una descripción de los bloques de datos que no registran información faltante, y la cantidad de datos que fueron estimados al final de cada bloque. Las bases de datos "limpias" fueron guardas en `data/02 clean` y les fue añadida la variable binaria **Filled** que indica si el renglón en cuestión fue estimado ($1$) o es original ($0$).



Por otro lado, en cuando a la calidad de datos existentes, se observa un comportamiento atípico en los primeros registros, causadopor la baja cantidad de operaciones en el mercado, esto se debe a que la plataforma Binance comenzó a operar a mediados del año 2017 y nuestro primer registro de datos se ubica en la fecha 2017-08-16. Con el propósito de eliminar el sesgo causado por la falta de operaciones en el mercado, se opta por restringir una fecha mínima para el análisis, la fecha elegida es el 2018-02-12 la cual puede ser modificada, antes de realizar la transformación de los datos, en el archivo `src/python/util/parameters.json`.

<figure>
    <img src="img/logtrades.jpg"  width="60%" />
    <figcaption>Al inicio del periodo se observa un reducido nivel de operaciones.</figcaption>
</figure>



# II Transformación de datos